In [7]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

GPU is available


In [8]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [02:14<00:00, 134.47s/ url]
0 examples [00:00, ? examples/s]

100%|█████████▉| 49908/50000 [00:00<00:00, 69560.34 examples/s]Dataset imdb_reviews downloaded and prepared to C:\Users\micha\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [11]:
hub_layer = hub.KerasLayer(
    "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1", 
    input_shape=[], 
    dtype=tf.string, 
    trainable=True
)

hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [12]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [14]:
history = model.fit(
    train_data.shuffle(10000).batch(512),
    epochs=20,
    validation_data=validation_data.batch(512),
    verbose=1
)

Epoch 1/20
30/30 [==============================] - 2s 77ms/step - loss: 0.7045 - accuracy: 0.5999 - val_loss: 0.6333 - val_accuracy: 0.6325
Epoch 2/20
30/30 [==============================] - 2s 67ms/step - loss: 0.6037 - accuracy: 0.6635 - val_loss: 0.5843 - val_accuracy: 0.6733
Epoch 3/20
30/30 [==============================] - 2s 70ms/step - loss: 0.5624 - accuracy: 0.6944 - val_loss: 0.5532 - val_accuracy: 0.7037
Epoch 4/20
30/30 [==============================] - 2s 65ms/step - loss: 0.5291 - accuracy: 0.7213 - val_loss: 0.5255 - val_accuracy: 0.7262
Epoch 5/20
30/30 [==============================] - 2s 67ms/step - loss: 0.4984 - accuracy: 0.7467 - val_loss: 0.5022 - val_accuracy: 0.7314
Epoch 6/20
30/30 [==============================] - 2s 65ms/step - loss: 0.4678 - accuracy: 0.7685 - val_loss: 0.4751 - val_accuracy: 0.7665
Epoch 7/20
30/30 [==============================] - 2s 66ms/step - loss: 0.4371 - accuracy: 0.7892 - val_loss: 0.4543 - val_accuracy: 0.7897
Epoch 8/20
30

In [15]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3208 - accuracy: 0.8582
loss: 0.321
accuracy: 0.858
